# Lane Finding Project for Self-Driving Car ND

In [38]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [39]:
import math

def grayscale(img):
    """
    Converts image to grayscale.
    """
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size):
    """
    Applies Gaussian blur to image.
    """
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
def canny(img, low_threshold, high_threshold):
    """
    Applies Canny transform.
    """
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Masks region outside of defined polygon.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """       
    Applies Hough transform and sorts lines.
    """
    left_lines = []
    right_lines = []
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

    # Separate left and right lines based on slope
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1))
            if slope < -0.5 and slope > -2:
                left_lines.append(line)
            elif slope > 0.5 and slope < 2:
                right_lines.append(line)
    
    return left_lines, right_lines

def coordinates(y_coor, lines, prev):
    """
    Calculates the x coordinates for a lane line.
    """
    if lines:
        lines = [line.flatten() for line in lines]
        
        # Calculate line lengths to be used as weights in weighted average
        lengths = [np.sqrt((line[2]-line[0])**2 + (line[3]-line[1])**2) for line in lines]
        
        # Calculate weighted average
        avg = [int(np.average(line, weights = lengths)) for line in zip(*lines)]
        
        # Calculate equation of line coefficients
        coef = np.polyfit((avg[0], avg[2]), (avg[1], avg[3]), 1)
        
        # Average equation of line coefficients with ones from previous frame
        if sum(prev) != 0:
            prev = [(prev[i] + coef[i])/2 for i in range(2)]
        elif sum(prev) == 0:
            prev = [coef[i] for i in range(2)]

    # Calculate x coordinates given y coordinates
    x_coor = [int((y-prev[1])/prev[0]) for y in y_coor]
    coor = list(zip(x_coor, y_coor))
    
    return coor, prev
        
def draw_lines(img, coor, color=[255, 0, 0], thickness=10):
    """
    Draws `lines` with `color` and `thickness`.
    """
    return cv2.line(img, (coor[0][0], coor[0][1]), (coor[1][0], coor[1][1]), color, thickness)

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    Combines transparency of input with initial image.
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Lane Line Identification - Images

In [40]:
# Variables
global left_prev
global right_prev

left_prev = [0, 0]
right_prev = [0, 0]

# Parameters
kernel_size = 3

low_threshold = 50
high_threshold = 150

rho = 2
theta = np.pi/180
threshold = 30
min_line_len = 100
max_line_gap = 150

for filename in os.listdir('images/input'):
    if filename.endswith('.jpg'):
        
        filepath = os.path.join('images/input/', filename)
        image = mpimg.imread(filepath)

        gray = grayscale(image)
        gray_blur = gaussian_blur(gray, kernel_size)
        edges = canny(gray_blur, low_threshold, high_threshold)
        
        # Polygon dimensions
        vertices = np.array([[(0, image.shape[0]),                    # Bottom left
                      (image.shape[1]/2-20, int(image.shape[0]*0.6)), # Top left
                      (image.shape[1]/2+20, int(image.shape[0]*0.6)), # Top right
                      (image.shape[1], image.shape[0])]],             # Bottom right
                    dtype = np.int32)
        edges_masked = region_of_interest(edges, vertices)
        
        # Identify lines in image and sort into left and right lines
        left_lines, right_lines = hough_lines(edges_masked, rho, theta, threshold, min_line_len, max_line_gap)
        
        # Calculate coordinates for left and right unbroken lines
        y_coor = [int(image.shape[0]*0.6), image.shape[0]]
        left_coor, left_prev = coordinates(y_coor, left_lines, left_prev)
        right_coor, right_prev = coordinates(y_coor, right_lines, right_prev)
        
        # Draw left and right unbroken lines
        line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
        draw_lines(line_image, left_coor) 
        draw_lines(line_image, right_coor)

        # Overlay drawn lines onto original image
        over = weighted_img(line_image, image)
        
        mpimg.imsave('images/output/identified_' + filename, over)

## Lane Line Identification - Videos

In [41]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [42]:
def process_image(image):

    # Variables
    global left_prev
    global right_prev

    # Parameters
    kernel_size = 3

    low_threshold = 50
    high_threshold = 150

    rho = 2
    theta = np.pi/180
    threshold = 30
    min_line_len = 100
    max_line_gap = 150

    gray = grayscale(image)
    gray_blur = gaussian_blur(gray, kernel_size)
    edges = canny(gray_blur, low_threshold, high_threshold)

    # Polygon dimensions
    vertices = np.array([[(0, image.shape[0]),                    # Bottom left
                  (image.shape[1]/2-20, int(image.shape[0]*0.6)), # Top left
                  (image.shape[1]/2+20, int(image.shape[0]*0.6)), # Top right
                  (image.shape[1], image.shape[0])]],             # Bottom right
                dtype = np.int32)
    edges_masked = region_of_interest(edges, vertices)

    # Identify lines in image and sort into left and right lines
    left_lines, right_lines = hough_lines(edges_masked, rho, theta, threshold, min_line_len, max_line_gap)

    # Calculate coordinates for left and right unbroken lines
    y_coor = [int(image.shape[0]*0.6), image.shape[0]]
    left_coor, left_prev = coordinates(y_coor, left_lines, left_prev)
    right_coor, right_prev = coordinates(y_coor, right_lines, right_prev)

    # Draw left and right unbroken lines
    line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    draw_lines(line_image, left_coor) 
    draw_lines(line_image, right_coor)

    return weighted_img(line_image, image)

### Solid White Lane Line

In [43]:
global left_prev
global right_prev

left_prev = [0, 0]
right_prev = [0, 0]

white_output = 'videos/output/identified_solidWhiteRight.mp4'
clip1 = VideoFileClip("videos/input/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video videos/output/identified_solidWhiteRight.mp4
[MoviePy] Writing video videos/output/identified_solidWhiteRight.mp4



100%|█████████▉| 221/222 [00:06<00:00, 36.20it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/output/identified_solidWhiteRight.mp4 

CPU times: user 2.67 s, sys: 972 ms, total: 3.64 s
Wall time: 6.82 s


In [44]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

### Solid Yellow Lane Line

In [45]:
global left_prev
global right_prev

left_prev = [0, 0]
right_prev = [0, 0]

yellow_output = 'videos/output/identified_solidYellowLeft.mp4'
clip2 = VideoFileClip('videos/input/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video videos/output/identified_solidYellowLeft.mp4
[MoviePy] Writing video videos/output/identified_solidYellowLeft.mp4



100%|█████████▉| 681/682 [00:18<00:00, 37.29it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/output/identified_solidYellowLeft.mp4 

CPU times: user 8.5 s, sys: 2.17 s, total: 10.7 s
Wall time: 19 s


In [46]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

### Challenge

In [47]:
global left_prev
global right_prev

left_prev = [0, 0]
right_prev = [0, 0]

challenge_output = 'videos/output/identified_challenge.mp4'
clip2 = VideoFileClip('videos/input/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video videos/output/identified_challenge.mp4
[MoviePy] Writing video videos/output/identified_challenge.mp4



100%|██████████| 251/251 [00:14<00:00, 18.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/output/identified_challenge.mp4 

CPU times: user 6.55 s, sys: 1.58 s, total: 8.12 s
Wall time: 15.8 s


In [48]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))